# Introduction to Research Module

This tutorial introduces Research functionality of batchflow.

Research class allows easily experimenting with models parameters and entire test and train workflow configurations as well as saving and loading results of experiments in a unified form.

## Roadmap

1. Introduction **<- You are here**
    * Basic example
        * 1 pipeline with fixed parameters
            * creating research
            * running several repetitions of an experiment
            * viewing research results
            * saving and loading research
    * Runnung experiments with different parameters aka grid
        * 1 pipeline with variable parameters
            * creating and viewing grids
            * viewing filtered research results
    * More complex execution strategies
        * 2 pipelines, train & test + function + grid
            * adding test pipeline
            * defining pipeline execution frequency
            * adding functions

1. Advanced topics
    * Reducing extra dataset loads
        * 1 pipeline with root and branch + grid
    * Performance
        * execution tasks managing
    * Cross-validation

1. Combining it all together
    * Super complex Research

## Basic example

We start with some useful imports and constant definitions

In [1]:
import sys
sys.path.append('../../..')

import matplotlib
%matplotlib inline

from batchflow import Pipeline, B, C, V
from batchflow.opensets import MNIST
from batchflow.models.tf import VGG7, VGG16
from batchflow.research import Research, Option

In [2]:
BATCH_SIZE=64
ITERATIONS=1000

We define a very simple pipeline that loads some MNIST data, and trains a VGG7 model on it. It also saves the loss on each iteration in a pipeline variable. Let's call it an experiment

We call a lazy version of pipeline's `run` method to define batch size to use. We pass `n_epochs=None`, because the duration of our experiment will be controlled by Research

In [3]:
mnist = MNIST()
train_root = mnist.train.p.run_later(BATCH_SIZE, shuffle=True, n_epochs=None)

model_config={
    'inputs/images/shape': (28, 28, 1),
    'inputs/labels/classes': 10,
    'inputs/labels/name': 'targets',
    'initial_block/inputs': 'images',
}

train_template = (Pipeline()
            .init_variable('loss')
            .init_model('dynamic', VGG7, 'conv', config=model_config)
            .to_array()
            .train_model('conv', 
                         images=B('images'), labels=B('labels'),
                         fetches='loss', save_to=V('loss', mode='w'))
)

train_ppl = train_root + train_template

The simpliest thing we can do with Research is running this experiment several times to see how loss'es dynamics changes from run to run.

To do this we define a Research object and add the pipeline to it. We call `add_pipeline` method and pass the `train_ppl` as the first parameter. The `variables` parameter gets a string or a list of strings that indicate which pipeline variables will be monitored by Research and written to research results on each iteration. We also provide `name` that will be written to results to indicate from where actually these results come.

`logging=True` adds additional logging of the pipeline execution to log file

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='loss', name='train_ppl', logging=True))

Now we run this Research.

We call a Research object's `run` method and we pass following parameters:
* `n_iters` - how many iterations will the experiment consist of. Each iteration here consists of processing a single batch
* `n_reps` - how many times we run our experiment
* `bar` - to provide a tqdm progress bar

In [ ]:
research.run(n_reps=4, n_iters=ITERATIONS, name='vgg7_research', bar=True)

Research vgg7_research is starting...
Distributor has 4 jobs with 1000 iterations. Totally: 4000


 69%|██████████████████████████████████████████████████████                        | 2773/4000 [40:31<17:55,  1.14it/s]

Each research is assigned with a `name` argument provided to `run`.
Results of the research and its log are saved in a folder with the same name in the working directory. 

In [ ]:
import os
for name in os.listdir():
    print("{}\t{}".format('Dir' if os.path.isdir(name) else 'File', name))

To see the experiments' results we call `load_result` method which returns a Pandas Dataframe. We can see that it has 4 collumns which contain 
* repetition - The number of experiment run
* iteration - the number of iteration
* name - the source of the variable
* loss - this is the variable name that we provided to `add_pipeline`

In [ ]:
results = research.load_results()
results.info()

In [ ]:
results.sample(10)

We can now draw a nice plot showing our loss dynamics on each experiment repetition

In [ ]:
results.pivot(index='iteration', columns='repetition', values='loss').plot()

## Introducing grids

We have learned to run multiple repetitions of a single experiment with Research. 

We can also run several experiments with different parameters in one research. Suppose we want to compare the performance of VGG7 and VGG16 models with different layouts ('convolution-normalization-activation' vs 'convolution-activation-normalization') without bias and we would also like to check VGG7 preformance with bias and default 'cna' layout.

To do so we define a grid of parameters as follows. 
We define an Option that consists of the parameter to vary and a list of values that we want to try in our research. Each parameter value defines a node in a parameter grid. We can add grids to unite the nodes and multiply grids to get Cartesian product.

`grid.gen_configs()` returns a generator that yields one node (that is, a single experiment specification) at a time. Printing a list of all nodes shows us all experiment modifications in a dict-like mode.

In [ ]:
grid = (Option('layout', ['cna', 'can']) * Option('model', [VGG7, VGG16]) * Option('bias', [False])
        +  Option('layout', ['cna']) * Option('bias', [True]) * Option('model', [VGG7]))
list(grid.gen_configs())

We should now update `model_config` so that it could read the values from the grid.

We pass named expressions as parameters values with names from our parameter grid. 
We define layout and bias in the model config and we pass model type as a named expression to `init_model` method of the pipeline (config option named expression `C()` should be used)

In [ ]:
model_config.update({
    'body/block/layout': C('layout'),
    'common/conv/use_bias': C('bias'),
})


# For reference: previous train_template definition 
#     train_template = (Pipeline()
#                 .init_variable('loss', init_on_each_run=list)
#                 .init_model('dynamic', VGG7, 'conv', config=model_config) # Note model class defined explicitly
#                 .to_array()
#                 .train_model('conv', 
#                              images=B('images'), labels=B('labels'),
#                              fetches='loss', save_to=V('loss', mode='w'))
#     )

train_template = (Pipeline()
            .init_variable('loss', init_on_each_run=list)
            .init_model('dynamic', C('model'), 'conv', config=model_config) # Model class defined via named expression
            .to_array()
            .train_model('conv', 
                         images=B('images'), labels=B('labels'),
                         fetches='loss', save_to=V('loss', mode='w'))
)

train_ppl = train_root + train_template

We define a new research as before but also add the grid of parameters with `add_grid` method. After that we run the research, and it takes much longer because we are now running 5 different experiments 2 times each. 

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='loss', name='train')
            .add_grid(grid))

research.run(n_reps=2, n_iters=ITERATIONS, name='vgg_layout_bias_research', bar=True)


Research results now contain new columns *layout*, *bias* and *model* with corresponding parameter values.

In [ ]:
results = research.load_results()
results.head()

#### Controlling *load_results* output

To combine different config options in a single-column string representation we can pass `use_alias=True` to `load_results` method.

In [ ]:
results = research.load_results(use_alias=True)
results.head()

This is very useful when comparing separate experiments  

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(3, 2, figsize=(15, 15))
for i, (config, df) in enumerate(results.groupby('config')):
    x, y = i//2, i%2
    df.pivot(index='iteration', columns='repetition', values='loss').plot(ax=ax[x, y])
    ax[x, y].set_title(config)
    ax[x, y].set_xlabel('iteration')
    ax[x, y].set_ylabel('loss')
    ax[x, y].grid(True)
    ax[x, y].legend()

We can filter the results to use certain parameter values.

In [ ]:
research.load_results(aliases={'model': 'VGG7'}).head()

The code below does effectively the same but when passing `config` we define actual parameter values (like model class), not their string representations

In [ ]:
research.load_results(configs={'model': VGG7}).head()

We can also get results corresponding to certain repetitions of experiments or certain iterations.

Here we have only one output variable - *loss* - but if we had many we could also load only some of them using `variables` parameter

In [ ]:
research.load_results(repetitions=1, iterations=[0,9], variables=['loss']).head()

### Loading saved researches 

<font color=red size=5>**BROKEN**</font>

As mentioned above, after each run of a research a folder with log information and results is created

In [ ]:
for name in os.listdir():
    print("{}\t{}".format('Dir' if os.path.isdir(name) else 'File', name))

We can use `Research.load` class method to load a previousely run research by its name

In [ ]:
#loaded_research = Research.load('vgg_layout_bias_research')

We can check its parameters...

In [ ]:
#loaded_research.describe()

... and run it one more time

In [ ]:
#loaded_research.run(n_reps=4, n_iters=ITERATIONS, name='vgg7_research_1_load', bar=True)

Note that `n_reps=4, name='vgg7_research_1_load'` from `run` arguments are ignored and the values from the loaded research are used

In [ ]:
#loaded_research.load_results().sample(10)

## More complex execution strategies

Usually we would like to run more than a single train pipeline.

Let's define a test pipeline to monitor a model's loss on test set while learning. 

We start with defining a train pipeline as before

In [ ]:
grid = Option('layout', ['cna', 'can'])

model_config={
    'inputs/images/shape': (28, 28, 1),
    'inputs/labels/classes': 10,
    'inputs/labels/name': 'targets',
    'initial_block/inputs': 'images',
    'body/block/layout': C('layout'),
}

train_template = (Pipeline()
            .init_variable('train_loss')
            .init_model('dynamic', VGG7, 'conv', config=model_config)
            .to_array()
            .train_model('conv', 
                         images=B('images'), labels=B('labels'),
                         fetches='loss', save_to=V('train_loss', mode='w'))
)

train_ppl = train_root + train_template

We want the test pipeline to be run on the whole test set from time to time during training. 

In order to get this, we specify lazy-run in test pipeline with `n_epochs=1` and we pass `run=True` to research's `add_pipeline`. These 2 parameter values tell Research to run the pipeline on the whole test set for 1 epoch, instead of running it batch-wise (which is how the test pipeline is run). We also pass `execute=100` parameter to `add_pipeline` to tell Research that this pipeline should be executed only on each 100-th iteration.

`execute` parameter defines frequency of pipeline execution. One can tell research to execute a pipeline periodically and pass an `int`, or to execute it on specific iteration by passing zero-based iteration number in following format: `'#{it_no}'` or by simply passing `'last'`. `execute` can also be a list, for example `execute=["#0', '#123', 100, 'last']` means that a pipeline will be executed on first iteration, 124-th iteration, every 100-th iteration and the last one.

To let the test pipeline to use the model trained by train pipeline we pass this model via `C('import_from')` named expression. We call `import_model` of test pipeline and pass `import_from='train_ppl'` parameter to `add_pipeline`, where `'train_ppl'` is the name of our train pipeline.

In [ ]:
test_root = mnist.test.p.run_later(BATCH_SIZE, shuffle=True, n_epochs=1) #Note  n_epochs=1
test_template = (Pipeline()
                 .init_variable('test_loss')
                 .import_model('conv', C('import_from'))
                 .to_array()
                 .predict_model('conv', 
                                images=B('images'), labels=B('labels'),
                                fetches='loss', save_to=V('test_loss', mode='w')))

test_ppl = test_root + test_template

In [ ]:
TEST_EXECUTE_FREQ = 100

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
            .add_pipeline(test_ppl, variables='test_loss', name='test_ppl',
                         execute=TEST_EXECUTE_FREQ, run=True, import_from='train_ppl') # Note run=True
            .add_grid(grid))

research.run(n_iters=ITERATIONS, name='train_test_research', bar=True)


Now our research results contain entries for test_ppl

In [ ]:
results = research.load_results()
results.sample(10)

## Functions

In order to perform some computations while running pipelines, for example to acquire values that are not produced by the model explicitely, one can use functions.

We will now add a function to calculate model's accuracy on test set

We redefine test pipeline removing `'test_loss'` and adding variables to store predictions and metrics computed for these predictions with `gather_metrics`

In [ ]:
test_template = (Pipeline()
                 .init_variable('predictions')
                 .init_variable('metrics')
                 .import_model('conv', C('import_from'))
                 .to_array()
                 .predict_model('conv', 
                                images=B('images'), labels=B('labels'),
                                fetches='predictions', save_to=V('predictions'))
                .gather_metrics('class', targets=B('labels'), predictions=V('predictions'), 
                                fmt='logits', axis=-1, save_to=V('metrics')))

test_ppl = test_root + test_template

We also define a function to be executed during experiment's iterations. Such functions take 2 required parameters: `iteration` and `experiment` which are fed to it by Research and optional keyword parameters which should be provided externally.

In [ ]:
def get_accuracy(iteration, experiment, pipeline):
    pipeline = experiment[pipeline].pipeline
    metrics = pipeline.get_variable('metrics')
    return metrics.evaluate('accuracy')

We define a new reseach. Note that we now don't ask the test pipeline to write any variables to results explicitely (we don't pass `variables` parameter to corresponding `add_pipeline`.

To add a function we call `add_function` method, providing it with a function object and`returns` parameters that specifies what should be written to research results.  We also pass keyword parameter `pipeline='test_ppl'` that will be substituted to `get_accuracy`.

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
            .add_pipeline(test_ppl, name='test_ppl',
                         execute=TEST_EXECUTE_FREQ, run=True, import_from='train_ppl')
            .add_function(get_accuracy, returns='accuracy', name='test_accuracy_fn', 
                          execute=TEST_EXECUTE_FREQ, pipeline='test_ppl')
            .add_grid(grid))

research.run(n_reps=1, n_iters=ITERATIONS, name='fun_research', bar=True)

Research results now contain accuracy from `'test_accuracy_fn'`

We can load results that come from a certain source by passing `names` parameter to `load_results`

In [ ]:
research.load_results(names='test_accuracy_fn').sample(5)

### gather_metrics

Actually collecting metrics is a very common case so there is a special method `get_metrics`. We pass pipeline name to collect metrics from (`pipeline`), the name of the named expression which stores collected metrics (`metrics_var`) and what to calculate(`metrics_name`). 

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
            .add_pipeline(test_ppl, name='test_ppl',
                         execute=TEST_EXECUTE_FREQ, run=True, import_from='train_ppl')
            .get_metrics(pipeline='test_ppl', metrics_var='metrics', metrics_name='accuracy',
                         returns='accuracy', execute=TEST_EXECUTE_FREQ)
            .add_grid(grid))

research.run(n_reps=1, n_iters=ITERATIONS, name='get_metrics_research', bar=True)

`get_metrics` implicitely adds a function with a name *\[pipeline_name\]_metrics* 

In [ ]:
research.load_results(names='test_ppl_metrics').sample(10)